<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [3]:
import urllib.request
target_url = "https://www.gutenberg.org/files/100/100-0.txt"
text = ""

data = urllib.request.urlopen(target_url)
for line in data:
    text = text + "\n" + str(line)

In [4]:
len(text)

6946939

In [5]:
# Split the text into runs of 40 characters each.
maxlen = 40
step = 3

sequences = [] # X 
next_chars = [] # Y

for i in range(0, len(text) - maxlen, step):
    sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('sequences:', len(sequences))

sequences: 2315633


In [6]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))       

In [7]:
import numpy as np

In [8]:
# One-hot encode the text
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char_indices[char]] = 1

    y[i, char_indices[next_chars[i]]] = 1

In [9]:
len(chars)

89

In [30]:
sequences[3016]

"s adore his beauty still,\\r\\n'\nb'Attendi"

In [10]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop

In [21]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metric=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import LambdaCallback

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [15]:
import numpy as np
import random
import sys
import os

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Train on 2315633 samples
Epoch 1/5
2315520/2315633 [============================>.] - ETA: 0s - loss: 1.3927
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "e all\r\n'
b'    recreants and dastards,"
e all\r\n'
b'    recreants and dastards, and the courses,\r\n'
b'    The streath of the command and the beard and the procellents with his for the true of means of the hands and the world\r\n'
b'    The part of the house and be the world and the world that the words,\r\n'
b'That the worses the worses the world of the bear.\r\n'
b'    The worse as the worse of the worsess and the heavents,\r\n'
b'That the man and the worses and way the s
----- diversity: 0.5
----- Generating with seed: "e all\r\n'
b'    recreants and dastards,"
e all\r\n'
b'    recreants and dastards, and where hath contrice,\r\n'
b'    The worsest many for in enours, and some and is for the with the troth and more haw more in some your though but good all the first course\r\n'
b'    Th

C:\Users\soobt\Anaconda3\envs\jupyterlab\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


0\x99lls\r\n'
b'As brumseon the sousines good many keep there come from his worne ariold\r\n'
b'    What gentle to kill ers put samandy\r\n'
b"    Coreland of well-man, what is my late.\r\n'
b"  KINe?\r\n'
b'Tloon an eyes the soul, hi
----- diversity: 1.2
----- Generating with seed: "e glory fills the world with loud report"
e glory fills the world with loud reportly;\r\n"
b"    Done got ye away and Yollubb'ans, no myy.\r\n'
b'    Thou faith.\r\n'
b'\r\n'
b' [_Rein! againio, herm lest orbur,\r\n'
b'As you, Gindaico-vance\xe2\x80\x99s? O Ineng ehpour\r\n'
b"    thus to a ttemen with therabl down in\r\n'
b"    Why cut Berul is you curusgiyy from royor, want;\r\n'
b'Dake him, welkey come, the eye, for I campar; and tear,\r\n'
b'    couservick bhoodhing-bhoad 4
2315633/2315633 [==============================] - 1083s 468us/sample - loss: 1.3437
Epoch 4/5
2315264/2315633 [============================>.] - ETA: 0s - loss: 1.5577
----- Generating text after Epoch: 3
----- diversity: 0.2
-----

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN